<a href="https://colab.research.google.com/github/Pingying-Chen/R-Samples/blob/main/Regressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple Regression Analysis with R

### Dataset
Basic Computer Data (https://www.kaggle.com/datasets/kingburrito666/basic-computer-data-set)
### Data Description
This is a simple dataset for basic data analysis.  
The Computers data set has 6259 observations with 10 variables. The variables are price (numeric), speed (numeric), hd (numeric), ram (numeric), screen (numeric), cd (categorical), multi (categorical), premium (categorical), ads (numeric), trend (numeric).  
### Problem Description
This notebook is a practice of using R to do different regressions to predict the price of the computer and find out what the best regression model is for this dataset.

## 1. Preparation

In [ ]:
# Install packages
install.packages("corrplot")
install.packages("dplyr")
install.packages("caret")
install.packages("car")
install.packages("magrittr")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

also installing the dependencies ‘nloptr’, ‘RcppEigen’, ‘pbkrtest’, ‘lme4’


Warning message in install.packages("car"):
“installation of package ‘nloptr’ had non-zero exit status”
Warning message in install.packages("car"):
“installation of package ‘RcppEigen’ had non-zero exit status”
Warning message in install.packages("car"):
“installation of package ‘lme4’ had non-zero exit status”
Warning message in install.packages("car"):
“installation of package ‘pbkrtest’ had non-zero exit status”
Warning message in install.packages("car"):
“installation of package ‘car’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

In [ ]:
# Load packages
library("corrplot")
library("dplyr")
library("caret")
library("car")
library("magrittr")

corrplot 0.92 loaded


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggplot2

Loading required package: lattice



ERROR: Error in library("car"): there is no package called ‘car’


## 2. Load Data

In [ ]:
computers <- read.csv("data/computers.csv")

In [ ]:
# Read first six rows
head(computers)

In [ ]:
# Remove first column
computers <- computers[, -1]

# Summary
summary(computers)

In [ ]:
# Turn characters to factors
computers[sapply(computers, is.character)] <- lapply(computers[sapply(computers, is.character)], as.factor)

str(computers)

## 3. Exploratory Analysis

### 3.1 Continuous values distribution

In [ ]:
par(mfrow=c(3,2))
hist(computers$price, main="Price Distribution", xlab="Price", col="skyblue")
hist(computers$speed, main="Speed Distribution", xlab="Speed", col="skyblue")
hist(computers$hd, main="HD Distribution", xlab="HD", col="skyblue")
hist(computers$ram, main="RAM Distribution", xlab="RAM", col="skyblue")
hist(computers$ads, main="Ads Distribution", xlab="Ads", col="skyblue")
hist(computers$trend, main="Trend Distribution", xlab="Trend", col="skyblue")

### 3.2 Categorical values

In [ ]:
par(mfrow=c(1,3))
barplot(table(computers$cd), main="CD Distribution", ylab="Count", col="lightgreen")
barplot(table(computers$multi), main="Multi Distribution", ylab="Count", col="lightgreen")
barplot(table(computers$premium), main="Premium Distribution", ylab="Count", col="lightgreen")

### 3.3 Continuous vs categorical

In [ ]:
par(mfrow=c(1,3))
boxplot(price ~ cd, data=computers, main="Price by cd", xlab="cd", ylab="Price", col="violet")
boxplot(price ~ multi, data=computers, main="Price by multi", xlab="multi", ylab="Price", col="violet")
boxplot(price ~ premium, data=computers, main="Price by Premium", xlab="Premium", ylab="Price", col="violet")

### 3.4 Correlation plot

In [ ]:
par(mfrow=c(1,1))
(cor_matrix <- cor(computers[, c("price", "speed", "hd", "ram", "screen", "ads", "trend")]))
corrplot(cor_matrix, method="circle")

## 4. Basic Multiple Regression

### 4.1 Training test set split and model create

In [ ]:
# Traning & test set split
set.seed(123)
training.samples1 <- computers$price %>%
  createDataPartition(p = 0.8, list = FALSE)
c_train1  <- computers[training.samples1, ]
c_test1 <- computers[-training.samples1, ]

# Create Model
lm1 <- lm(price ~ ., data = c_train1)
summary(lm1)

#### Interpretation:
- The model is statistically significant with a very low p-value (< 2.2e-16), which means the predictors, as a set, significantly relate to the response variable price.
- The p-values of all coefficients are smaller than 0.001, which means the coefficients of the intercept and all predictive factors are statistically significant.
- The model can be written as: price = 265.25463 + 9.23583 * speed + 0.77610 * hd + 48.47308 * ram + 125.20002 * screen + 56.52204 * cdyes + 98.17646 * multiyes − 499.50670 * premiumyes + 0.68946 * ads − 51.50423 * trend.

### 4.2 Diagnostic plots

In [ ]:
par(mfrow=c(2,2))
plot(lm1)

#### Interpretation:
- **Residuals vs. Fitted:** There’s a slight non-linearity. The non-constant variance indicates potential
heteroscedasticity.
- **Q-Q Plot:** The residuals seem to deviate slightly from the normal distribution, especially in the tails.
- **Scale-Location:** Similar to the Residuals vs. Fitted plot, suggesting potential heteroscedasticity.
- **Residuals vs. Leverage:** Some points have high leverage, but they don’t appear to be highly influential
in terms of Cook’s distance.
  
The diagnostic plots suggest some potential violations of regression assumptions, especially regarding linearity
and homoscedasticity. Addressing these issues may further improve the model.

### 4.3 Model Performance

In [ ]:
# Make Predictions
prediction1 <- lm1 %>% predict(c_test1)

# Model Performance
data.frame(RMSE = RMSE(prediction1, c_test1$price),
           R2 = R2(prediction1, c_test1$price))

#### Interpretation:
- **R-squared:** The model explains approximately 77.26% of the variance in the price. This suggests a reasonably good fit.
- **RMSE:** The root mean square error is 267.75, indicating on average, the model’s predictions are approximately 267.75 units away from the actual values.

## 5. Multiple Regression with Interactions

### 5.1 Training test set split and model create

In [ ]:
# Check correlations
(cor_matrix <- cor(computers[, c("speed", "hd", "ram", "screen", "ads", "trend")]))

In [ ]:
# Traning & test set split
set.seed(123)
training.samples2 <- computers$price %>%
  createDataPartition(p = 0.8, list = FALSE)
c_train2  <- computers[training.samples2, ]
c_test2 <- computers[-training.samples2, ]

# Create Model
lm2 <- lm(price ~ speed * hd * ram + screen + cd + multi + premium + ads + trend, data = c_train2)
summary(lm2)

#### Interpretation:
- The model is statistically significant with a very low p-value (< 2.2e-16), which means the predictors, as a set, significantly relate to the response variable price.
- The p-values of all coefficients except for speed:ram are smaller than 0.001, which means the coefficients of the intercept and all predictive factors are statistically significant. The p-value of speed:ram coefficient is between 0.05 to 0.1, suggesting it not as significant as others.
- The model can be written as: price = −72.70 + 16.53 * speed + 2.228 * hd + 70.23 * ram + 120.4 * screen + 44.26 * cdyes + 102.5 * multiyes − 549.8 * premiumyes + 0.4382 * ads − 51.09 * trend − 0.02029 * speed:hd − 0.1389 * speed:ram − 0.05573 * hd:ram + 0.0005235 * speed:hd:ram

### 5.2 Diagnostic plots

In [ ]:
par(mfrow=c(2,2))
plot(lm2)

#### Interpretation:
- **Residuals vs. Fitted:** Slight U-shape suggests potential non-linearity in the data.
- **Q-Q Plot:** The points closely follow the line, indicating that the residuals are approximately normally distributed.
- **Scale-Location:** The spread looks reasonably constant, suggesting homoscedasticity (equal variances) of the residuals.
- **Residuals vs. Leverage:** No observation has high leverage and high residual simultaneously. Though some points have high Cook’s distance, suggesting they might be influential.
  
In summary, the model fits reasonably well, but there’s a hint of non-linearity. There are some potential influential points, but they don’t seem to unduly influence the model’s fit. Further investigation might be needed to handle the non-linearity and influential points.



### 5.3 Model Performance

In [ ]:
# Make Predictions
prediction2 <- lm2 %>% predict(c_test2)

# Model Performance
data.frame(RMSE = RMSE(prediction2, c_test2$price),
           R2 = R2(prediction2, c_test2$price))

#### Interpretation:
- **R-squared:** The model explains approximately 81.37% of the variance in the price. This suggests a reasonably good fit.
- **RMSE:** The root mean square error is 250.41, indicating on average, the model’s predictions are approximately 250.41 units away from the actual values.

## 6. Multiple Regression with Multicollinearity

### 6.1 Training test set split and model create

In [ ]:
vif(lm1)

In [ ]:
# Traning & test set split
set.seed(123)
training.samples3 <- computers$price %>%
  createDataPartition(p = 0.8, list = FALSE)
c_train3  <- computers[training.samples3, ]
c_test3 <- computers[-training.samples3, ]

# Create Model
lm3 <- lm(price ~. -hd, data = c_train3)
summary(lm3)

#### Interpretation:
- The model is statistically significant with a very low p-value (< 2.2e-16), which means the predictors, as a set, significantly relate to the response variable price.
- The p-values of all coefficients except for the intercept are smaller than 0.001, which means the coefficients of the intercept and all predictive factors are statistically significant. The p-value of the intercept coefficient is between 0.001 to 0.01, suggesting it not as significant as others.
- The model can be written as: price = 224.18378 + 9.79021 ∗ speed + 70.55952 ∗ ram + 127.56138 ∗ screen + 88.26443 ∗ cdyes + 71.48390 ∗ multiyes − 474.02904 ∗ premiumyes + 0.47647 ∗ ads − 43.25888 ∗ trend.

### 6.2 Diagnostic plots

In [ ]:
par(mfrow=c(2,2))
plot(lm3)

#### Interpretation:
- **Residuals vs. Fitted:** The plot shows a somewhat even distribution of residuals around the red line, but with a noticeable curve. This suggests potential non-linearity in the data.
- **Q-Q Plot:** The points closely follow the straight line, suggesting the residuals are nearly normally distributed. However, there are deviations at the tails, implying potential outliers.
- **Scale-Location:** The spread of residuals is mostly consistent across the range of fitted values, although there’s a slight fan shape. This suggests that the assumption of homoscedasticity (equal variance) is mostly met, with minor deviations.
- **Residuals vs. Leverage:** Most data points have low leverage. The red line denotes Cook’s distance, and few points lie above it, which might be influential points worth further investigation.
Overall, the model seems to fit the data decently but might benefit from further examination of influential points and potential non-linearities.

### 6.3 Model Performance

In [ ]:
# Make Predictions
prediction3 <- lm3 %>% predict(c_test3)

# Model Performance
data.frame(RMSE = RMSE(prediction3, c_test3$price),
           R2 = R2(prediction3, c_test3$price))

## 7. Stepwise Regression

### 7.1 Training test set split and model create

In [ ]:
# Traning & test set split
set.seed(123)
training.samples4 <- computers$price %>%
  createDataPartition(p = 0.8, list = FALSE)
c_train4  <- computers[training.samples4, ]
c_test4 <- computers[-training.samples4, ]

# Define intercept-only model
intercept_only <- lm(price ~ 1, data = c_train4)

# Define model with all predictors
all <- lm(price ~ ., data = c_train4)

# Create model
lm4 <- step(intercept_only, direction = 'both', scope=formula(all), trace=0)
summary(lm4)

In [ ]:
lm4$anova

#### Interpretation:
The model is basically the same as the first model.  
The slight differences of the coefficients between these two models are caused by different training test set split.

### 7.2 Diagnostic plots

In [ ]:
# Diagnostic plots
par(mfrow=c(2,2))
plot(lm4)

### 7.3 Model Performance

In [ ]:
# Make Predictions
prediction4 <- lm4 %>% predict(c_test4)

# Model Performance
data.frame(RMSE = RMSE(prediction4, c_test4$price),
           R2 = R2(prediction4, c_test4$price))

## 8. Model Comparison

The second regression, **the multiple regression with interaction** should be the **best** model considering the high adjusted R-squared value and lower RMSE.
However, we should also consider the non-linearity in this model. Further investigation might be needed.